In [1]:
import cv2
import numpy as np
import time

In [2]:
# Initialize video capture (camera feed or video file)
capture_video = cv2.VideoCapture(0) 
	
# Allow the camera to warm up 
time.sleep(1)
count = 0
background = 0

# Capturing the background in range of 60 
# (capture a few seconds of background to stabilize the image)
for i in range(60): 
    return_val, background = capture_video.read() 
    if return_val == False: 
        continue

background = np.flip(background, axis = 1)  # Flipping the frame for consistency

# Start reading from the video
while (capture_video.isOpened()): 
    return_val, img = capture_video.read() 
    if not return_val: 
        break
    count = count + 1
    img = np.flip(img, axis = 1)  # Flip for a mirror view

    # Convert the image from BGR to HSV for better color detection
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # ------------------------------------ BLUE DETECTION ------------------------------------- #
    # Adjusting the HSV range to detect blue color
    lower_blue = np.array([100, 100, 100])  # Lower range for blue
    upper_blue = np.array([140, 255, 255])  # Upper range for blue
    mask1 = cv2.inRange(hsv, lower_blue, upper_blue)

    # Refining the mask corresponding to the detected blue color
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8), iterations=2) 
    mask1 = cv2.dilate(mask1, np.ones((3, 3), np.uint8), iterations=1) 
    mask2 = cv2.bitwise_not(mask1)

    # Generating the final output where blue areas are replaced by background
    res1 = cv2.bitwise_and(background, background, mask=mask1) 
    res2 = cv2.bitwise_and(img, img, mask=mask2) 
    final_output = cv2.addWeighted(res1, 1, res2, 1, 0) 

    # Display the final output
    cv2.imshow("Invisible Blue", final_output) 
    k = cv2.waitKey(10) 
    if k == 27:  # Press 'ESC' to exit
        break

# Release the video capture and close all windows
capture_video.release() 
cv2.destroyAllWindows()